# ===========================================
# 2_data_preprocessing.ipynb
# ===========================================

# Zelle 1: Bibliotheken importieren
import pandas as pd
import re
import os
import nltk
from nltk.tokenize import word_tokenize
import json
from sklearn.model_selection import train_test_split

# NLTK-Token-Daten herunterladen, falls noch nicht geschehen
nltk.download('punkt')

# Wir definieren Pfade zum "raw"- und "processed"-Ordner
RAW_DATA_DIR = "../data/raw"
PROCESSED_DATA_DIR = "../data/processed"

# ===========================================
# Zelle 2: Rohdaten laden
# ===========================================
# Beispiel: Wir haben eine CSV-Datei mit einer Spalte "text".
# (Pass an, falls du mehrere Dateien einlesen willst)

csv_file_path = os.path.join(RAW_DATA_DIR, "sample_texts.csv")
df = pd.read_csv(csv_file_path)

print(f"Anzahl Datensätze: {len(df)}")
print("Erste Zeilen:")
display(df.head())

# ===========================================
# Zelle 3: Datensäuberung
# ===========================================
# Beispielhafte Säuberung: Entfernen von HTML-Tags, mehrfachen Leerzeichen, etc.

def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    # HTML-Tags entfernen
    text = re.sub(r"<.*?>", "", text)
    # Mehrfache Leerzeichen reduzieren
    text = re.sub(r"\s+", " ", text)
    # Trim
    text = text.strip()
    return text

df["cleaned_text"] = df["text"].apply(clean_text)

# Optional: Entferne sehr kurze Texte oder null-Werte
df.dropna(subset=["cleaned_text"], inplace=True)
df = df[df["cleaned_text"].str.len() > 0]

print("\nBeispiel für gesäuberten Text:")
display(df["cleaned_text"].head(5))

# ===========================================
# Zelle 4: Tokenisierung
# ===========================================
# Wir verwenden NLTK's word_tokenize zum Zerlegen in Wort-Token.

def tokenize_text(text):
    tokens = word_tokenize(text)
    # Optional: Kleinbuchstaben
    tokens = [t.lower() for t in tokens]
    return tokens

df["tokens"] = df["cleaned_text"].apply(tokenize_text)

print("\nBeispiel für Tokens:")
display(df["tokens"].head(5))

# ===========================================
# Zelle 5: Vokabular erstellen
# ===========================================
# Wir bauen ein Frequenz-Dict, um die häufigsten Tokens zu ermitteln.
# Dann werden Tokens in IDs gemappt.

from collections import Counter

all_tokens = []
for token_list in df["tokens"]:
    all_tokens.extend(token_list)

token_freqs = Counter(all_tokens)
print(f"Anzahl eindeutiger Tokens: {len(token_freqs)}")

# Beispiel: Wir beschränken das Vokabular auf die top-N häufigsten Tokens
# (falls du sehr große Daten hast)
VOCAB_SIZE = 20000  # oder len(token_freqs), wenn du alles behalten willst
most_common_tokens = token_freqs.most_common(VOCAB_SIZE)

# Wir definieren eine Start-ID ab 4, weil wir 0,1,2,3 z.B. für <PAD>, <UNK>, <BOS>, <EOS> reservieren können
word2id = {
    "<PAD>": 0,
    "<UNK>": 1,
    "<BOS>": 2,
    "<EOS>": 3
}
idx = 4

for token, freq in most_common_tokens:
    if token not in word2id:
        word2id[token] = idx
        idx += 1

print(f"Vokabulargröße (inkl. Sondertokens): {len(word2id)}")

# Umgekehrtes Mapping (ID->Wort)
id2word = {v: k for k, v in word2id.items()}

# ===========================================
# Zelle 6: Token-IDs generieren
# ===========================================
# Wir ersetzen in jedem Tokensatz die Wörter durch ihre IDs
# Tokens, die nicht im Vokabular sind, bekommen <UNK>-ID

UNK_ID = word2id["<UNK>"]
BOS_ID = word2id["<BOS>"]
EOS_ID = word2id["<EOS>"]

def tokens_to_ids(token_list, word2id, unk_id=UNK_ID):
    return [word2id[t] if t in word2id else unk_id for t in token_list]

df["token_ids"] = df["tokens"].apply(lambda tk: tokens_to_ids(tk, word2id))

print("\nBeispiel für token_ids:")
display(df["token_ids"].head(5))

# ===========================================
# Zelle 7: Erstellung von Sequenzen (Train/Val/Test-Splits)
# ===========================================
# Wir gehen davon aus, dass wir später in src/dataset.py Sequenzen bilden.
# Hier zeigen wir exemplarisch, wie du train/val/test trennst.

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# ===========================================
# Zelle 8: Speichern der Ergebnisse
# ===========================================
# Jetzt speichern wir unser Vokabular und die Datenframes mit token_ids in "processed".

# 1) Vokabular
vocab_path = os.path.join(PROCESSED_DATA_DIR, "vocab.json")
with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump(word2id, f, ensure_ascii=False)

# 2) Train/Val/Test CSV (oder Parquet, JSON, etc.)
train_path = os.path.join(PROCESSED_DATA_DIR, "train.csv")
val_path = os.path.join(PROCESSED_DATA_DIR, "val.csv")
test_path = os.path.join(PROCESSED_DATA_DIR, "test.csv")

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("\nDaten gespeichert in:")
print(train_path)
print(val_path)
print(test_path)
print("Vokabular gespeichert in:", vocab_path)

# ===========================================
# Zelle 9: Ausblick
# ===========================================
# An dieser Stelle haben wir:
#  - Die Rohdaten bereinigt
#  - Tokens erstellt
#  - Ein Vokabular generiert & ID-Mappings
#  - Einfache Train/Val/Test-Splits angelegt
#  - Alles in data/processed/ gespeichert

print("\nNächster Schritt: In '3_training_demo.ipynb' oder in src/train.py können wir nun das LSTM-Modell trainieren.")
